In [54]:
import pandas as pd
import re
from collections import Counter

In [14]:
df = pd.read_csv('../../data/Evaluation_CoTs/final.csv')

In [75]:
# Compile a regex pattern to match any of the common terms
common_terms = ["a mistake", "an error", "not solvable", "apologize" ,"not enough information","that's still not it..."]

# Compile a regex pattern to match any of the common terms
pattern = re.compile(r'\b(?:' + '|'.join(common_terms) + r')\b', re.IGNORECASE)

# Function to find and extract matched terms with surrounding words
def extract_surrounding_terms(text, pattern, window_size=10):
    words = re.findall(r'\b\w+\b', text)
    for match in re.finditer(pattern, text):
        matched_term = match.group(0)
        
        start = match.start()
        end = match.end()

        # Get the surrounding words within the specified window size
        start_idx = max(start - window_size, 0)
        end_idx = max(end + window_size + 1, len(words))
        surrounding_text = text[start_idx:end_idx]
        
        yield (matched_term, surrounding_text)

In [76]:
set(df.Model)

{'claude-3-haiku-20240307', 'gpt-3.5-turbo-0125', 'gpt-4', 'llama3_cleaned'}

In [77]:
results = {
    'gpt-4':[],
    'gpt-3.5-turbo-0125':[],
    'claude-3-haiku-20240307':[],
    'llama3_cleaned':[]
}
for row_idx in range(len(df)):
    row = df.iloc[row_idx]
    for i in range(40):
        CoT_column = f"CoT_{i}"
        string = str(row[CoT_column]).replace('\n',';')
        for match in extract_surrounding_terms(string, pattern):
            results[row.Model].append(match[0])

In [78]:
for key,val in results.items():
    print(key,':')
    print(Counter(val))
    print('\n\n')

gpt-4 :
Counter({'a mistake': 977, 'an error': 17, 'not enough information': 2, 'An error': 1})



gpt-3.5-turbo-0125 :
Counter({'an error': 257, 'a mistake': 164, 'An error': 39, 'Not enough information': 3, 'not solvable': 2, 'not enough information': 2})



claude-3-haiku-20240307 :
Counter({'a mistake': 356, 'not enough information': 168, 'an error': 97, 'Not enough information': 8, 'not solvable': 3})



llama3_cleaned :
Counter({'a mistake': 1738, 'an error': 1639, 'apologize': 109, 'An error': 23, 'Not enough information': 18, 'not enough information': 15, 'not solvable': 1, 'Not Enough Information': 1})
